# HR Analytics : Which features is most important?


Let's find out which features is most important then others to predect Job change.

I do not ML approach, it is handy, but test data's target answers are not provided.
But use tree decision model instead.

## Loading library

In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) 
        
import warnings
warnings.filterwarnings("ignore")        
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns


## Loading Data

In [ ]:
aug_train = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
aug_test = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_test.csv')
aug_train.info()

## Change Data : Reads nicer & Converting objects to integers

I refer Josh's works on below code.
https://www.kaggle.com/joshuaswords/awesome-hr-data-visualization-prediction

In [ ]:
aug_train['last_new_job'] = aug_train['last_new_job'].apply(lambda x: 'Never' if x == 'never' else x) #just reads nicer
aug_train['enrolled_university'][aug_train['enrolled_university'] == 'no_enrollment'] = 'No Enrollment' #just reads nicer
aug_train['company_size'] = aug_train['company_size'].apply(lambda x: '10-49' if x == '10/49' else x) #diff replacement method

aug_train['experience'] = aug_train['experience'].apply(lambda x: '0' if x == '<1' else x)
aug_train['experience'] = aug_train['experience'].apply(lambda x: '20' if x == '>20' else x)


aug_train['company_size'].fillna('0',inplace=True)
aug_train['company_type'].fillna('Unknown',inplace=True)
aug_train['major_discipline'].fillna('Unknown',inplace=True)
aug_train['gender'].fillna('Not provided',inplace=True)


# Repeat steps on Test set

aug_test['last_new_job'] = aug_test['last_new_job'].apply(lambda x: 'Never' if x == 'never' else x) #just reads nicer
aug_test['enrolled_university'][aug_test['enrolled_university'] == 'no_enrollment'] = 'No Enrollment' #just reads nicer
aug_test['company_size'] = aug_test['company_size'].apply(lambda x: '10-49' if x == '10/49' else x) #diff replacement method

aug_test['experience'] = aug_test['experience'].apply(lambda x: '0' if x == '<1' else x)
aug_test['experience'] = aug_test['experience'].apply(lambda x: '20' if x == '>20' else x)

aug_test['company_size'].fillna('0',inplace=True)
aug_test['company_type'].fillna('Unknown',inplace=True)
aug_test['major_discipline'].fillna('Unknown',inplace=True)
aug_test['gender'].fillna('Not provided',inplace=True)



# This is what I mean with respect to the gender categories:

aug_train['target'].groupby(aug_train['gender']).mean()

# I will drop the remaining missing data
aug_train.dropna(inplace=True)
aug_test.dropna(inplace=True)

# Converting objects to integers

aug_train['experience'] = aug_train['experience'].astype(str).astype(int)
aug_test['experience'] = aug_test['experience'].astype(str).astype(int)

aug_train['training_hours'] = aug_train['training_hours'].astype(str).astype(int)
aug_test['training_hours'] = aug_test['training_hours'].astype(str).astype(int)

## One-Hot on Categorical data

In [ ]:
aug_train_one_hot = pd.get_dummies(aug_train, columns = ['city',
                                                         'gender',
                                                         'relevent_experience',
                                                         'enrolled_university',
                                                         'education_level',
                                                         'major_discipline',
                                                         'company_size',
                                                         'company_type',
                                                         'last_new_job'])

train_input = aug_train_one_hot.drop(['enrollee_id','target'], axis=1).to_numpy()
train_target = aug_train_one_hot['target'].to_numpy()
print(train_input.shape, train_target.shape)

### Get columns list for tree graph (later)

In [ ]:
columns_list = aug_train_one_hot.drop(['enrollee_id','target'], axis=1).columns.values.tolist()

#### You might get better result when using StandardScaler, but for easy reading and understanding, I use original distribution.
#### Leave the code for reference : 

In [ ]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# ss.fit(train_input)
# train_input = ss.transform(train_input)

### Apply DecisionTreeClassifier for find out key factor

max_depth = 2 is enough, no score gain when use 3.
if you use max_depth more than 4, you may get higher score, but it looks like over-fitting.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=2, random_state=42)
dt.fit(train_input, train_target)
print(dt.score(train_input, train_target))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(20,10))
plot_tree(dt, filled=True, feature_names=columns_list, fontsize=15)
plt.show()

### Only one factor has great importance == City Development Index

In [ ]:
print(dt.feature_importances_)

### Let's check what is relationship between City Development Index and other factores
#### 1 = Job changed
#### 0 = Job unchanged
#### Training_hours did not has great importance, because it shows almost same distribution on CDI 0.9 and 0.6.

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")

g = sns.jointplot(x="training_hours", y="city_development_index", data=aug_train,
                  kind="kde", 
                  hue = "target",
                  color="b", height=10)

# Then... what is City Development Index ?

City Development Index :
https://en.wikipedia.org/wiki/City_development_index

I can get the data from pp32-34, Table B. CDI and component subindices, GLOBAL URBAN INDICATORS DATABASE, Version 2 , Global Urban Observatory, United Nations Human Settlements Programme (UN - Habitat), ISBN 92-1-131627- 8 

Note :
* Index No. 46 is missing on original data
* Total cases on the data == 161 cities
* City Development Index = (Infrastructure index + Waste index + Education index + Health index + City Product index) / 5 
* But math was not mathched, because Each sub-index is a combination of several indicators that have been normalized to give a value between 0 and 1. 

In [ ]:
CDI = pd.DataFrame(
    {'City':['Abidjan', 'Accra', 'Antananarivo', 'Bangui', 'Banjul', 'Brazzaville', 'Bujumbura', 'Bulawayo', 'Chegutu', 'Conakry', 'Douala', 'Entebbe', 
             'Gweru', 'Harare', 'Ibadan', 'Jinja', 'Kigali', 'Kinshasa', 'Kisumu', 'Kumasi', 'Lagos', 'Lilongwe', 'Lome', 'Mombasa', 'Monrovia', 'Nairobi', 'Niamey', 
             'Nouakchott', 'Aden', 'Algiers', 'Amman', 'Baghdad', 'Casablanca', 'Damascus', 'Doha', 'Gaza', 'Kuwait', 'Muscat', 'Rabat', 'Sana\'a', 'Tripoli', 
             'Tunis', 'Apia', 'Bangalore', 'Bangkok', 'Cebu', 'Chiang Mai', 'Chittagong', 'Colombo', 'Dhaka', 'Hanam', 'Hanoi', 'Hohhot', 'Hong Kong', 'Jakarta', 
             'Kathmandu', 'Lahore', 'Mandaluyong', 'Medan', 'Naga', 'Penang', 'Phnom Penh', 'Port Moresby', 'Pusan', 'Semarang', 'Seoul', 'Singapore', 'Surabaya', 
             'Suva', 'Sylhet', 'Tangail', 'Ulaanbaatar', 'Vientiane', 'Yangon', 'Amal', 'Ankara', 'Aversa', 'Basel', 'Hull', 'Madrid', 'Melbourne', 'Pamplona', 'Stockholm', 
             'Umea', 'Armenia', 'Asuncion', 'Belem', 'Belize', 'Bridgetown', 'Buenos Aires', 'Cajamarca', 'Cienfuegos', 'Ciudad Habana', 'Ciudad Juarez', 'Colon', 'Cordoba', 
             'Cuenca', 'Gran Concepcion', 'Guayaquil', 'Huanuco', 'Huaras', 'Icapui', 'Iquitos', 'Kingston', 'Leon', 'Lima', 'Manta', 'Maranguape', 'Marinilla', 'Medellin', 
             'Montego Bay', 'Montevideo', 'Port of Spain', 'Porto Alegre', 'Puyo', 'Quetzaltenango', 'Quito', 'Recife', 'Rio de Janeiro', 'Rosario', 'San Salvador', 'Santa Clara', 
             'Santa Cruz de la Sierra', 'Santiago de Chile', 'Santiago de los Caballeros', 'Santo Andre', 'Tacna', 'Tena', 'Tome', 'Tumbes', 'Valparaiso', 'Vina del mar', 
             'Astrakhan', 'Belgorod', 'Belgrad', 'Bishkek', 'Bourgas', 'Brno', 'Bydgoszcz', 'Chisinau', 'Gdansk', 'Katowice', 'Kostroma', 'Ljubljana', 'Moscow', 
             'Nizhny Novgorod', 'Novomoscowsk', 'Omsk', 'Poznan', 'Prague', 'Pushkin', 'Riga', 'Sarajevo', 'Sofia', 'Surgut', 'Tallinn', 'Tbilisi', 'Troyan', 
             'Veliko Tarnovo', 'Veliky Novgorod', 'Vilnius'],
     'Country':['Côte d\'Ivoire', 'Ghana', 'Madagascar', 'Central African Republic', 'Gambia', 'Congo', 'Burundi', 'Zimbabwe', 'Zimbabwe', 'Guinea', 'Cameroon', 'Uganda', 
                'Zimbabwe', 'Zimbabwe', 'Nigeria', 'Uganda', 'Rwanda', 'Congo, Dem. Rep.', 'Kenya', 'Ghana', 'Nigeria', 'Malawi', 'Togo', 'Kenya', 'Liberia', 'Kenya', 
                'Niger', 'Mauritania', 'Yemen', 'Algeria', 'Jordan', 'Iraq', 'Morocco', 'Syria', 'Qatar', 'Palestine', 'Kuwait', 'Oman', 'Morocco', 'Yemen', 'Libya', 
                'Tunisia', 'Samoa, Western', 'India', 'Thailand', 'Philippines', 'Thailand', 'Bangladesh', 'Sri Lanka', 'Bangladesh', 'Korea, Rep. Of', 'Vietnam', 
                'Central African Republic', 'China', 'Indonesia', 'Nepal', 'Pakistan', 'Philippines', 'Indonesia', 'Philippines', 'Malaysia', 'Cambodia', 'Papua New Guinea', 
                'Korea, Rep. Of', 'Indonesia', 'Korea, Rep. Of', 'Singapore', 'Indonesia', 'Fiji', 'Bangladesh', 'Bangladesh', 'Mongolia', 'Laos', 'Myanmar', 'Sweden', 
                'Turkey', 'Italy', 'Switzerland', 'Canada', 'Spain', 'Australia', 'Spain', 'Sweden', 'Sweden', 'Colombia', 'Paraguay', 'Brazil', 'Belize', 'Barbados', 
                'Argentina', 'Peru', 'Cuba', 'Cuba', 'Mexico', 'Panama', 'Argentina', 'Ecuador', 'Chile', 'Ecuador', 'Peru', 'Peru', 'Brazil', 'Peru', 'Jamaica', 
                'Nicaragua', 'Peru', 'Ecuador', 'Brazil', 'Colombia', 'Colombia', 'Jamaica', 'Uruguay', 'Trinidad and Tobago', 'Brazil', 'Ecuador', 'Guatemala', 
                'Ecuador', 'Brazil', 'Brazil', 'Argentina', 'El Salvador', 'Cuba', 'Bolivia', 'Chile', 'Dominican Republic', 'Brazil', 'Peru', 'Ecuador', 
                'Chile', 'Peru', 'Chile', 'Chile', 'Russian Federation', 'Russian Federation', 'Yugoslavia', 'Kyrgyzstan', 'Bulgaria', 'Czech Republic', 
                'Poland', 'Moldova', 'Poland', 'Poland', 'Russian Federation', 'Slovenia', 'Russian Federation', 'Russian Federation', 'Russian Federation', 
                'Russian Federation', 'Poland', 'Czech Republic', 'Russian Federation', 'Latvia', 'Bosnia and Herzegovina', 'Bulgaria', 'Russian Federation', 'Estonia', 
                'Georgia', 'Bulgaria', 'Bulgaria', 'Russian Federation', 'Lithuania'],
     'Region':['Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 
               'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 'Africa', 
               'Arab States', 'Arab States', 'Arab States', 'Arab States', 'Arab States', 'Arab States', 'Arab States', 'Arab States', 'Arab States', 'Arab States', 
               'Arab States', 'Arab States', 'Arab States', 'Arab States', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 
               'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 
               'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 
               'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'Asia-Pacific', 'HIC', 'HIC', 'HIC', 'HIC', 'HIC', 'HIC', 'HIC', 'HIC', 'HIC', 
               'HIC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 
               'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 'LAC', 
               'LAC', 'LAC', 'LAC', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 
               'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 
               'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional', 'Transitional' ],
     'CDI':[39.7, 46.6, 34.5, 27.5, 40.5, 27.1, 43.1, 74.6, 55.4, 37.1, 48.2, 39.7, 68.5, 70.2, 27.3, 42.7, 31.9, 35.3, 53.5, 57.1, 29.3, 31.5, 40.9, 53.4, 28.1, 56.1, 
            21.7, 30.8, 42.5, 76.1, 77, 47.2, 57.2, 64.3, 82.6, 44.5, 88.3, 76.7, 63.1, 41.8, 67.7, 74.6, 59, 58, 82.6, 67, 78.5, 39.3, 58.4, 47.1, 89.9, 74.2, 65.8, 
            92, 69.2, 62, 61.1, 70.8, 58, 66.7, 67.3, 39.2, 39.3, 88.6, 58.1, 95.8, 94.5, 62.2, 69.3, 40.6, 35.3, 66, 47.1, 51.3, 94.7, 72, 81.7, 95.2, 98, 94.4, 95.5, 
            90.2, 97.3, 95.8, 54, 55.7, 53, 63, 77.9, 79.4, 72.5, 69.6, 71, 72.6, 66.2, 81.9, 76.5, 79.9, 69, 59.5, 41.9, 35.9, 35.9, 62.7, 48.9, 67.2, 52.1, 62.1, 66.4, 
            73, 60.1, 72.5, 72.9, 68.5, 55.2, 57.8, 60.2, 69.9, 79.4, 72.6, 70.2, 72.2, 64.3, 79.2, 68, 76.8, 57.9, 52.6, 80.3, 50.5, 87.8, 87.4, 71.2, 77.3, 69.6, 
            55.8, 85.4, 89.6, 71, 76.9, 77.4, 76.4, 75.1, 91.7, 89.9, 78.6, 74.5, 73.8, 79.8, 89.6, 81.1, 81.1, 68.1, 79.1, 77.6, 81.5, 72.2, 64.8, 71.5, 76.2, 83.3],
     'City Product':[56.6, 49.4, 44.4, 42, 46, 30.7, 50.3, 67.5, 55.1, 50.8, 46.6, 42.7, 58.9, 68.1, 39.8, 41.8, 45.9, 64.6, 60, 42.8, 42.1, 34.4, 55.4, 54.6, 46.6, 
                     63.2, 40, 50, 42.3, 69.2, 65.1, 65.1, 64.5, 57.9, 89.6, 63.7, 92.8, 77.5, 62.7, 40, 70.2, 70.6, 65.3, 51.1, 77.7, 70.2, 68, 55.4, 46.9, 57.5, 
                     87.9, 59.6, 60, 89.4, 66.2, 60.4, 71.1, 69.6, 40.8, 70.3, 73.4, 40.1, 69, 83.1, 47, 94.7, 91.6, 58.6, 73.5, 51.4, 48.7, 49.9, 44, 47.4, 81.2, 
                     73, 82.2, 91.5, 97.2, 87.9, 90, 84, 93.5, 85.9, 63.6, 75.7, 51.6, 64.3, 87.1, 82.2, 58.4, 62.6, 64.9, 75.4, 58.4, 77.6, 50.5, 81.5, 61.1, 36.2, 
                     43.6, 40.1, 51.2, 63, 52.8, 64.4, 35.1, 51.6, 62, 61.6, 61.2, 81.5, 73.4, 74.8, 58.8, 64.3, 57.6, 68.2, 82.3, 70, 72.5, 82.2, 55.8, 74.9, 
                     65.7, 51.4, 50, 35.1, 74.8, 40.4, 74.8, 74.2, 59.3, 69.7, 69, 64, 61.6, 79.9, 72.3, 54, 73.8, 79, 64.5, 83.4, 81, 69.1, 59.1, 67.7, 79.1, 83, 
                     69.5, 71.8, 56.1, 70.9, 82, 78, 58.6, 66.9, 64.2, 67.5, 75.2],
     'Infrastructure':[21.7, 50, 22.5, 14.9, 16, 31.6, 32.8, 84.5, 45, 30.4, 34.9, 29.5, 83.8, 82.5, 21.1, 42, 29.8, 34.9, 42, 47.5, 29.5, 33, 21.8, 41.3, 4.7, 
                       52.5, 22, 12.3, 40.2, 75, 84.9, 56.3, 81.8, 68.6, 90, 65, 99, 78, 65.5, 39, 73.1, 61, 63.5, 82.7, 89.6, 42.3, 82.5, 41, 68.6, 48.8, 84.8, 
                       72, 67.9, 99.3, 57.3, 76.3, 78.5, 66.4, 54.6, 55.4, 79.3, 40.1, 18.1, 89.3, 34.8, 98.4, 99.5, 64.2, 79.3, 40.4, 28.3, 60.8, 58, 65.3, 98.8, 
                       93.9, 96.3, 99.7, 98.8, 98.8, 99.8, 93.5, 99.5, 98.8, 84, 39.5, 50.5, 58.8, 69.9, 92.1, 62.8, 70.4, 74.8, 76.8, 58.8, 79.5, 83.5, 88.6, 
                       63.8, 49.3, 30, 52.8, 69.4, 66.2, 45.7, 67.7, 65, 47, 88.9, 96.2, 59.8, 87.9, 87.5, 77.8, 65, 58.8, 76.6, 64.8, 86.2, 83.3, 82.3, 70, 
                       60.7, 92.8, 79, 93, 53.1, 69, 74.8, 50, 87.3, 89.3, 77.8, 82.5, 91.7, 42.9, 95.8, 90.9, 91.7, 94.5, 87.1, 92.2, 79.6, 99.3, 98.7, 90, 
                       88.5, 78.8, 94.2, 98.4, 96.8, 89.5, 78.8, 93.7, 86.7, 95.4, 88.2, 81.5, 98, 86.2, 88.9],
     'Waste':[29, 0, 0, 0.1, 48, 0.4, 18.2, 75.5, 38.5, 40, 39.5, 16, 70.5, 96.5, 2.5, 34, 10, 13.3, 32.5, 49, 2, 0, 0, 52.3, 0, 39, 0, 2, 17.5, 80, 77.2, 0, 
              5, 37, 70, 0, 93, 80, 52.5, 17.5, 27.5, 85, 0, 31.3, 74.5, 66, 85, 0.3, 45, 0, 90.4, 90, 50.1, 99, 46.7, 37.5, 50, 45.9, 42.6, 43, 20, 0, 10, 
              84.7, 37.2, 99.3, 100, 35, 37.5, 0.3, 0, 53, 0, 0, 100, 40.4, 45.6, 100, 100, 100, 100, 89.5, 100, 100, 0, 2, 0, 45, 53.5, 50, 78.5, 41.1, 
              50, 48.5, 45, 74.6, 87.5, 52.9, 51.7, 50, 0, 0, 32, 20, 0, 34, 0, 49, 50, 42.5, 15, 17.1, 40, 49.8, 0, 32.5, 0, 54.5, 62.6, 36.3, 40.6, 35, 58.5, 
              51.7, 40, 65, 32, 2.5, 75.4, 35, 100, 95, 46, 50.5, 10.4, 7.5, 96.5, 100, 14.3, 71.2, 51.8, 34.3, 54.9, 99, 86.8, 53.7, 50, 44.5, 48.2, 88.5, 
              55, 45, 75, 58.5, 46.6, 61.8, 47.5, 4.5, 28, 49, 77],
     'Health':[94.6, 94, 92.7, 90.2, 87.8, 86.4, 82.6, 78.4, 84.1, 85.2, 87.5, 81.1, 67.7, 80.6, 86.6, 86.6, 81.2, 81.6, 80.5, 80.7, 26.4, 76, 81, 83.8, 78.8, 
               84.6, 78.3, 74.1, 94.5, 88.8, 94.9, 88.8, 88.9, 85.9, 82, 80.9, 82.9, 77.9, 83.6, 76.3, 70.2, 42.4, 94.1, 42.2, 90.6, 59.1, 90, 83.1, 61.5, 84.6, 
               80.2, 35, 75.7, 61.5, 79, 40.7, 52.9, 29, 31.8, 44, 82.3, 86.3, 43.9, 71, 69.6, 81.8, 64.8, 78.5, 54.5, 77.5, 80.5, 47.2, 65.9, 54.7, 93.7, 93.9, 
               93.3, 86.8, 82.9, 80.2, 50.5, 74.2, 75.6, 56.5, 90.9, 83.8, 90.2, 87.2, 89.3, 93.6, 89.9, 80.9, 90.7, 84.3, 89.2, 83, 80.3, 82.3, 85.4, 83.4, 81.3, 
               82, 86.3, 80.5, 84.6, 89.3, 85.4, 82, 51.6, 72.5, 81.4, 88.3, 82.3, 90.1, 78.4, 90.1, 77.8, 86.1, 76.5, 79.2, 68.8, 86.5, 84.1, 60.3, 78.4, 83.4, 
               70.1, 60.4, 62.9, 71.4, 32.5, 61.5, 90.6, 90.3, 88.7, 87.3, 92.5, 84.7, 90, 86.2, 85.6, 89.3, 87.6, 86.9, 85.6, 86.4, 81.5, 80.5, 77.8, 78.5, 64.9, 
               77.8, 61, 53.7, 73.8, 86, 64.6, 62.3, 65.9, 87, 53.9 ], 
     'Education':[42.4, 62, 52.5, 36.8, 37.8, 18.5, 57.7, 77.6, 77.6, 29.5, 59.8, 68.2, 77.6, 77.6, 29.1, 63.1, 44.8, 22.8, 68.3, 62, 29.1, 58.3, 73.3, 65.3, 36.5, 
                  65.3, 14.9, 39, 46.6, 75.2, 75, 51.9, 48.6, 75.8, 77.2, 51.6, 70, 63.4, 48.6, 46.6, 86.6, 70.8, 82.7, 48.5, 78.9, 78.7, 76.3, 38.1, 45.3, 59.2, 
                  97.7, 69, 69.5, 81.3, 95.7, 64.8, 40.8, 89.3, 72.7, 80.2, 75.8, 49, 40.2, 97.9, 93.5, 97.7, 88.6, 75.4, 75.5, 49.3, 38.1, 92, 71.3, 73.4, 99.8, 
                  72.5, 90.7, 89.8, 99.5, 91.3, 94.1, 90.6, 99.8, 99.8, 40.5, 78.1, 84.3, 64.7, 88.6, 88.4, 84.1, 84.7, 84.7, 80.4, 82.2, 88.4, 79.3, 86.7, 83.3, 84.1, 
                  48.8, 11, 37.7, 74.5, 65.5, 84.1, 82.8, 84.3, 52.7, 81.8, 74.5, 89.4, 76.9, 58.2, 82.8, 57.2, 88.3, 86.1, 84.3, 87, 70.9, 84.7, 72.4, 86.7, 76.4, 
                  88.7, 85.5, 82.8, 86.7, 41.3, 86.7, 88.2, 92.4, 98.7, 90.1, 86.2, 86, 86.8, 89.3, 84.3, 85.2, 88.9, 94.2, 86.2, 99.3, 97.3, 94.8, 94.4, 88.6, 
                  87.4, 99.8, 87.2, 47, 86.3, 88.3, 89.8, 85.5, 90.5, 85.6, 97.5, 88.3]  })


# Conclusion
### On HR Analytics data, trainees of these Cities would change their current job :.

In [ ]:
pd.set_option('display.max_rows', None)
CDI[CDI['CDI'] < 62.5].sort_values(by="CDI", ascending=False)